# CEFA 2016 EE dataset
This notebook is used to create the dataset for CEFA 2016 EE. The dataset consists of multiple files. 
- Q_EE_HH_Results_General
- GADM_level_0
- GADM_level_1
- GADM_level_2


In [42]:
import os
import sys
import pandas as pd
import numpy as np

sys.path.append("../../")  # Adds higher directory to python modules path.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
from core.ODEDataset import ODEDataset
from utils import common_modifiers,  appliances_modifiers, geospatial

import pathlib
from utils import constants


In [43]:
DB_ROOT = "../../playground/data/CEFA_Tanzania/2016_EE"
root = pathlib.Path(DB_ROOT)

ID_COL = "ID"

## Clusters
Clusters are group of columns used together to create a new feature. 


## Categories
Map the original values to the standard values.


In [58]:
Education_level_original2final = {'P': 'Primary education',
                                    'S': 'Lower secondary education',
                                    'U': 'Bachelor\'s or equivalent level'}

Socio_status_original2final = {
    'Agr': 'Own-account worker farm',
    'Bre': 'Own-account worker farm',
    'Self': 'Own-account worker non-farm',
    'Sal': 'Employee non-farm',
}

Tariff_payment_frequency_original2final = {
    'W': 'Weekly',
    'M': 'Monthly',
    'M3': 'Every 3 months',
}


## Q_EE_HH_Results_General

### Merge Q_EE_HH_Results_General, Q_EE_PS_Results_General, Q_EE_US_Results_General

In [61]:
Q_EE_HH_Results_General = ODEDataset("Q_EE_HH_Results_General")
Q_EE_PS_Results_General = ODEDataset("Q_EE_PS_Results_General")
Q_EE_US_Results_General = ODEDataset("Q_EE_US_Results_General")

Q_EE_HH_Results_General.from_excel(root.joinpath("Q_EE_HH_Results_General.xlsx"))

Q_EE_HH_Results_General = Q_EE_HH_Results_General.apply(common_modifiers.rename({
    "Q_EE_HH": ID_COL,
    "HH_size": "size",
    "HH_rooms": "rooms",
    "HH_members": "members",
}))

# Columns present in PS and US
Q_EE_HH_Results_General = Q_EE_HH_Results_General.apply(common_modifiers.add_const_driver_many({
    "Class": np.nan,
    "Role": np.nan,
    "number": np.nan,
    "Consumption": np.nan,
}))

Q_EE_PS_Results_General.from_excel(root.joinpath("Q_EE_PS_Results_General.xlsx"))
Q_EE_PS_Results_General = Q_EE_PS_Results_General.apply(common_modifiers.rename({
    "Q_EE_PS": ID_COL,
    "PS_size": "size",
    "PS_number": "number",
}))

# Columns  present in HH
Q_EE_PS_Results_General = Q_EE_PS_Results_General.apply(common_modifiers.add_const_driver_many({
    'members': np.nan,
    'rooms': np.nan,
    'Income_source': np.nan,
    'Income': np.nan,
    'Expenditures': np.nan,
}))

Q_EE_US_Results_General.from_excel(root.joinpath("Q_EE_US_Results_General.xlsx"))
Q_EE_US_Results_General = Q_EE_US_Results_General.apply(common_modifiers.rename({
    "Q_EE_US": ID_COL,
    "US_size": "size",
    "US_number": "number"
}))

# Columns  present in HH
Q_EE_US_Results_General = Q_EE_US_Results_General.apply(common_modifiers.add_const_driver_many({
    'members': np.nan,
    'rooms': np.nan,
    'Income_source': np.nan,
    'Income': np.nan,
    'Expenditures': np.nan,
}))

Q_Results_General = Q_EE_HH_Results_General.concat(Q_EE_PS_Results_General).concat(Q_EE_US_Results_General)
Q_Results_General = Q_Results_General.group_by(ID_COL)
Q_Results_General.preview(2000)

,ID,Village,Interviewee,Age,Sex,Education,members,rooms,size,Income_source,Income,Expenditures,Bill_schedule,Bill,Comments,Class,Role,number,Consumption
0,Q_EE_HH_01,[Nyombo],[Anastasia Mbata],[44],[F],[P],[5.0],[10.0],[100.0],[Agr],[30000.0],[10000.0],[M],[48000],[Frequent cut-offs],[nan],[nan],[nan],[nan]
1,Q_EE_HH_02,[Nyombo],[Chongolo],[42],[M],[P],[6.0],[7.0],[35.0],[Agr],[100000.0],[100000.0],[M],[30000],[Frequent cut-offs; Slow repairing service],[nan],[nan],[nan],[nan]
2,Q_EE_HH_03,[Nyombo],[Bahat],[35],[M],[P],[6.0],[4.0],[20.0],[Agr],[20000.0],[20000.0],[M],[10000],[Frequent cut-offs],[nan],[nan],[nan],[nan]
3,Q_EE_HH_04,[Nyombo],[Nyahi],[32],[F],[P],[6.0],[7.0],[50.0],[Bre],[100000.0],[100000.0],[M],[20000],[Nothing],[nan],[nan],[nan],[nan]
4,Q_EE_HH_05,[Nyombo],[Mwinami],[30],[F],[P],[2.0],[4.0],[30.0],[Self],[100000.0],[40000.0],[M],[12000],[Don't know/don't answer],[nan],[nan],[nan],[nan]
5,Q_EE_HH_06,[Nyombo],[Jesca],[37],[F],[P],[7.0],[7.0],[80.0],[Agr],[30000.0],[30000.0],[M],[17000],[Frequent cut-offs],[nan],[nan],[nan],[nan]
6,Q_EE_HH_07,[Nyombo],[Kaduma],[38],[M],[P],[5.0],[8.0],[70.0],[Agr],[35000.0],[30000.0],[M],[20000],[Don't know/don't answer],[nan],[nan],[nan],[nan]
7,Q_EE_HH_08,[Nyombo],[Leonadi],[45],[M],[P],[6.0],[3.0],[30.0],[Agr],[30000.0],[25000.0],[M],[7000],[Electricity cost; Frequent cut-offs],[nan],[nan],[nan],[nan]
8,Q_EE_HH_09,[Nyombo],[Rosalia],[38],[F],[P],[4.0],[8.0],[50.0],[Agr],[200000.0],[100000.0],[W],[40000],[Frequent cut-offs],[nan],[nan],[nan],[nan]
9,Q_EE_HH_10,[Nyombo],[Jared],[35],[M],[P],[3.0],[7.0],[60.0],[Agr],[60000.0],[35000.0],[M],[15000],[Frequent cut-offs],[nan],[nan],[nan],[nan]


### GeoSpatial Information

In [62]:
gadm_level_1_df = pd.read_excel(root.joinpath("GADM_level_1.xlsx"))
gadm_level_2_df = pd.read_excel(root.joinpath("GADM_level_2.xlsx"))
gadm_level_3_df = pd.read_excel(root.joinpath("GADM_level_3.xlsx"))

village_gps_info_df = pd.read_excel(root.joinpath("Village_GPS_info.xlsx"))

In [63]:
Q_Results_General = Q_Results_General.new_feature("Township/Village",
                                                  lambda row: row["Village"][0])

Q_Results_General = Q_Results_General.new_feature("GADM_level_1",
                                                  geospatial.gis_info_by_village_level("GADM_level_1",
                                                                                       village_gps_info_df))

Q_Results_General = Q_Results_General.new_feature("GADM_level_2",
                                                  geospatial.gis_info_by_village_level("GADM_level_2",
                                                                                       village_gps_info_df))

Q_Results_General = Q_Results_General.new_feature("GADM_level_3",
                                                  geospatial.gis_info_by_village_level("GADM_level_3",
                                                                                       village_gps_info_df))

Q_Results_General = Q_Results_General.new_feature("Climate_zone_lev_1",
                                                  geospatial.gis_info_by_gadm_level('Climate_majority',
                                                                                    gadm_level_1_df,
                                                                                    'GADM_level_1'))
Q_Results_General = Q_Results_General.new_feature("Climate_zone_lev_2",
                                                  geospatial.gis_info_by_gadm_level('Climate_majority',
                                                                                    gadm_level_2_df,
                                                                                    'GADM_level_2'))


In [64]:
Q_Results_General = Q_Results_General.apply(common_modifiers.rename({
    "rooms": "Number_of_rooms"
})).apply(common_modifiers.take("Number_of_rooms", 0))

Q_Results_General = Q_Results_General.apply(common_modifiers.rename({
    "Income": "Income_of_household"
})).apply(common_modifiers.take("Income_of_household", 0))

Q_Results_General = Q_Results_General.apply(common_modifiers.rename({
    "Expenditures": "Monthly_expenditure"
})).apply(common_modifiers.take("Monthly_expenditure", 0))


Q_Results_General = Q_Results_General.new_feature("Number_adults",
                                                  lambda x: x["members"][0] * 0.535)

Q_Results_General = Q_Results_General.new_feature("Socio_status_HHH",
                                                  common_modifiers.categorize("Income_source",
                                                                              Socio_status_original2final)).apply(
    common_modifiers.take("Socio_status_HHH", 0))

Q_Results_General = Q_Results_General.new_feature("Education_level_HHH",
                                                  common_modifiers.categorize("Education",
                                                                              Education_level_original2final)).apply(
    common_modifiers.take("Education_level_HHH", 0))

Q_Results_General = Q_Results_General.apply(common_modifiers.add_const_driver_many({
    "Measurement_age": 2,
    "Connection_type": "National grid",  # needed to extract Monthly_expenditure
}))


Q_Results_General = Q_Results_General.new_feature("Tariff_payment_frequency",
                                                  common_modifiers.categorize("Bill_schedule",
                                                                              Tariff_payment_frequency_original2final)).apply(
    common_modifiers.take("Tariff_payment_frequency", 0))

Q_Results_General.preview()

,ID,Village,Interviewee,Age,Sex,Education,members,Number_of_rooms,size,Income_source,...,GADM_level_2,GADM_level_3,Climate_zone_lev_1,Climate_zone_lev_2,Number_adults,Socio_status_HHH,Education_level_HHH,Measurement_age,Connection_type,Tariff_payment_frequency
0,Q_EE_HH_01,[Nyombo],[Anastasia Mbata],[44],[F],[P],[5.0],10.0,[100.0],[Agr],...,Njombe,Ikuna,12,12,2.675,Own-account worker farm,Primary education,2,National grid,Monthly
1,Q_EE_HH_02,[Nyombo],[Chongolo],[42],[M],[P],[6.0],7.0,[35.0],[Agr],...,Njombe,Ikuna,12,12,3.210,Own-account worker farm,Primary education,2,National grid,Monthly
2,Q_EE_HH_03,[Nyombo],[Bahat],[35],[M],[P],[6.0],4.0,[20.0],[Agr],...,Njombe,Ikuna,12,12,3.210,Own-account worker farm,Primary education,2,National grid,Monthly
3,Q_EE_HH_04,[Nyombo],[Nyahi],[32],[F],[P],[6.0],7.0,[50.0],[Bre],...,Njombe,Ikuna,12,12,3.210,Own-account worker farm,Primary education,2,National grid,Monthly
4,Q_EE_HH_05,[Nyombo],[Mwinami],[30],[F],[P],[2.0],4.0,[30.0],[Self],...,Njombe,Ikuna,12,12,1.070,Own-account worker non-farm,Primary education,2,National grid,Monthly


In [65]:
Q_Results_General = Q_Results_General.select(
    [ID_COL, 'Monthly_expenditure',
     'Socio_status_HHH', 'Number_of_rooms', 'Number_adults',
     'Measurement_age', 'Tariff_payment_frequency',
     'Education_level_HHH', 'Income_of_household',
     'Climate_zone_lev_1', 'Climate_zone_lev_2'])


In [66]:
Q_Results_General.preview()

,ID,Monthly_expenditure,Socio_status_HHH,Number_of_rooms,Number_adults,Measurement_age,Tariff_payment_frequency,Education_level_HHH,Income_of_household,Climate_zone_lev_1,Climate_zone_lev_2
0,Q_EE_HH_01,10000.0,Own-account worker farm,10.0,2.675,2,Monthly,Primary education,30000.0,12,12
1,Q_EE_HH_02,100000.0,Own-account worker farm,7.0,3.210,2,Monthly,Primary education,100000.0,12,12
2,Q_EE_HH_03,20000.0,Own-account worker farm,4.0,3.210,2,Monthly,Primary education,20000.0,12,12
3,Q_EE_HH_04,100000.0,Own-account worker farm,7.0,3.210,2,Monthly,Primary education,100000.0,12,12
4,Q_EE_HH_05,40000.0,Own-account worker non-farm,4.0,1.070,2,Monthly,Primary education,100000.0,12,12


## Appliances
General Appliances sections
- Presence_phone_charger
- Presence_iron
- Presence_TV
- Presence_refrigerator/freezer
- Presence_radio/stereo
- Presence_DVD_player
- Presence_fan


### Merge Q_EE_HH_Results_Appliances, Q_EE_PS_Results_Appliances, Q_EE_US_Results_Appliances

In [67]:

Q_EE_HH_Results_Appliances = ODEDataset("Q_EE_HH_Results_Appliances")
Q_EE_PS_Results_Appliances = ODEDataset("Q_EE_PS_Results_Appliances")
Q_EE_US_Results_Appliances = ODEDataset("Q_EE_US_Results_Appliances")

Q_EE_HH_Results_Appliances.from_excel(root.joinpath("Q_EE_HH_Results_Appliances.xlsx"))
Q_EE_PS_Results_Appliances.from_excel(root.joinpath("Q_EE_PS_Results_Appliances.xlsx"))
Q_EE_US_Results_Appliances.from_excel(root.joinpath("Q_EE_US_Results_Appliances.xlsx"))


Q_EE_HH_Results_Appliances.drop_columns(['Comments_old'])
Q_EE_PS_Results_Appliances.drop_columns(['Comments_old'])


# 
# 
Q_EE_HH_Results_Appliances = Q_EE_HH_Results_Appliances.apply(common_modifiers.rename({
    "Q_EE_HH": ID_COL,
})).group_by(ID_COL)


Q_EE_PS_Results_Appliances = Q_EE_PS_Results_Appliances.apply(common_modifiers.rename({
    "Q_EE_PS": ID_COL,
})).group_by(ID_COL)


Q_EE_US_Results_Appliances = Q_EE_US_Results_Appliances.apply(common_modifiers.rename({
    "Q_EE_US": ID_COL,
})).group_by(ID_COL)



In [68]:
print("HH Appliances")
print("Not in PS", [i for i in Q_EE_HH_Results_Appliances.get_columns() if i not in Q_EE_PS_Results_Appliances.get_columns()])
print("Not in US", [i for i in Q_EE_HH_Results_Appliances.get_columns() if i not in Q_EE_US_Results_Appliances.get_columns()])
print("PS Appliances")
print("Not in HH", [i for i in Q_EE_PS_Results_Appliances.get_columns() if i not in Q_EE_HH_Results_Appliances.get_columns()])
print("Not in US", [i for i in Q_EE_PS_Results_Appliances.get_columns() if i not in Q_EE_US_Results_Appliances.get_columns()])
print("US Appliances")
print("Not in HH", [i for i in Q_EE_US_Results_Appliances.get_columns() if i not in Q_EE_HH_Results_Appliances.get_columns()])
print("Not in PS", [i for i in Q_EE_US_Results_Appliances.get_columns() if i not in Q_EE_PS_Results_Appliances.get_columns()])

HH Appliances
Not in PS []
Not in US []
PS Appliances
Not in HH []
Not in US []
US Appliances
Not in HH []
Not in PS []


In [69]:
Appliances = Q_EE_HH_Results_Appliances.concat(Q_EE_PS_Results_Appliances).concat(Q_EE_US_Results_Appliances)

Appliances.preview()

,ID,Item,Item_power,Item_number,Time_average,Time_minimum,ST_FW1,ET_FW1,ST_FW2,ET_FW2,Comments
0,Q_EE_HH_01,"[Charger, Ind_lights, Out_lights, Radio, TV]","[5, 18, 18, 10, 80]","[4, 5, 2, 1, 1]","[240, 540, 540, 180, 180]","[60, 540, 540, 10, 10]","[1080, 0, 0, 1080, 1080]","[1260, 360, 360, 1260, 1260]","[0, 1080, 1080, 1440, 0]","[0, 1440, 1440, 0, 0]","[nan, nan, nan, nan, nan]"
1,Q_EE_HH_02,"[Charger, Ind_lights, Out_lights, Radio, TV]","[5, 18, 18, 10, 80]","[2, 7, 3, 1, 1]","[360, 120, 660, 120, 180]","[360, 120, 660, 120, 180]","[0, 1080, 0, 1080, 1080]","[360, 1260, 360, 1440, 1440]","[1260, 0, 1080, 0, 0]","[1440, 0, 1440, 0, 0]","[nan, nan, nan, nan, nan]"
2,Q_EE_HH_03,"[Charger, Ind_lights, Out_lights, Radio, TV]","[5, 18, 18, 10, 80]","[2, 4, 1, 1, 1]","[90, 180, 660, 480, 180]","[90, 180, 660, 480, 180]","[360, 1080, 0, 0, 1080]","[540, 1440, 360, 360, 1260]","[0, 0, 1080, 1080, 0]","[0, 0, 1440, 1440, 0]","[nan, nan, nan, nan, nan]"
3,Q_EE_HH_04,"[Charger, Ind_lights]","[5, 18]","[2, 7]","[30, 660]","[30, 660]","[1080, 0]","[1440, 360]","[0, 1080]","[0, 1440]","[nan, nan]"
4,Q_EE_HH_05,"[Charger, Ind_lights, Iron, Out_lights, Radio,...","[5, 18, 1200, 18, 10, 80]","[3, 4, 1, 2, 1, 1]","[240, 360, 30, 660, 180, 360]","[90, 60, 30, 660, 180, 60]","[0, 0, 360, 0, 1080, 360]","[360, 360, 540, 360, 1440, 540]","[1080, 1080, 0, 1080, 0, 1080]","[1440, 1260, 0, 1440, 0, 1260]","[nan, nan, 1.0, nan, nan, nan]"


In [70]:
Appliances = Appliances.new_feature("Presence_phone_charger",appliances_modifiers.presence_appliances("Item", "Charger"))

Appliances = Appliances.new_feature("Presence_iron",appliances_modifiers.presence_appliances("Item", "Iron"))

Appliances = Appliances.new_feature("Presence_TV",appliances_modifiers.presence_appliances("Item", "TV"))

Appliances = Appliances.new_feature("Presence_refrigerator/freezer",
                                    appliances_modifiers.presence_appliances("Item", "Refrigerator"))


Appliances = Appliances.new_feature("Presence_stereo", lambda x: 0)
Appliances = Appliances.new_feature("Presence_radio", appliances_modifiers.presence_appliances("Item", "Radio"))
Appliances = Appliances.new_feature("Presence_radio/stereo",
                                    common_modifiers.multi_unify_presence(
                                        ['Presence_radio', 'Presence_stereo']))

Appliances = Appliances.new_feature("Presence_DVD_player", lambda  x: 0)
Appliances = Appliances.new_feature("Presence_fan", lambda  x: 0)

Appliances = Appliances.select([
    ID_COL, "Presence_phone_charger", "Presence_iron", "Presence_TV", "Presence_refrigerator/freezer",
    "Presence_radio/stereo", "Presence_DVD_player", "Presence_fan"
])

Appliances.preview()



,ID,Presence_phone_charger,Presence_iron,Presence_TV,Presence_refrigerator/freezer,Presence_radio/stereo,Presence_DVD_player,Presence_fan
0,Q_EE_HH_01,1,0,1,0,1,0,0
1,Q_EE_HH_02,1,0,1,0,1,0,0
2,Q_EE_HH_03,1,0,1,0,1,0,0
3,Q_EE_HH_04,1,0,0,0,0,0,0
4,Q_EE_HH_05,1,1,1,0,1,0,0


In [71]:
CEFA_2016_EE = Q_Results_General.merge(Appliances, ID_COL)

CEFA_2016_EE = CEFA_2016_EE.apply(common_modifiers.rename({
    ID_COL: "ID"
}))
CEFA_2016_EE = CEFA_2016_EE.apply(common_modifiers.add_const_driver_many({
    "Hours_available_electricity": np.nan,
    "Years_of_HHH_in_community": np.nan,
    'Dwelling_quality_index': np.nan,
    'Age_HHH': np.nan,
    'HH_with_home_business': np.nan,
    'Ownership_motorized_vehicle': np.nan,
    'Ownership_small_livestock': np.nan,
    'Clean_fuel': np.nan,
    'Ownership_large_livestock': np.nan,
}))
CEFA_2016_EE = CEFA_2016_EE.select(["ID"] + constants.DRIVERS_LIST + constants.PRESENCE_LIST)
CEFA_2016_EE.preview()

,ID,Years_of_HHH_in_community,Dwelling_quality_index,Hours_available_electricity,Measurement_age,Monthly_expenditure,Number_of_rooms,Climate_zone_lev_1,Climate_zone_lev_2,Age_HHH,...,Ownership_large_livestock,Clean_fuel,Tariff_payment_frequency,Presence_refrigerator/freezer,Presence_iron,Presence_fan,Presence_DVD_player,Presence_radio/stereo,Presence_phone_charger,Presence_TV
0,Q_EE_HH_01,NaN,NaN,NaN,2,10000.0,10.0,12,12,NaN,...,NaN,NaN,Monthly,0,0,0,0,1,1,1
1,Q_EE_HH_02,NaN,NaN,NaN,2,100000.0,7.0,12,12,NaN,...,NaN,NaN,Monthly,0,0,0,0,1,1,1
2,Q_EE_HH_03,NaN,NaN,NaN,2,20000.0,4.0,12,12,NaN,...,NaN,NaN,Monthly,0,0,0,0,1,1,1
3,Q_EE_HH_04,NaN,NaN,NaN,2,100000.0,7.0,12,12,NaN,...,NaN,NaN,Monthly,0,0,0,0,0,1,0
4,Q_EE_HH_05,NaN,NaN,NaN,2,40000.0,4.0,12,12,NaN,...,NaN,NaN,Monthly,0,1,0,0,1,1,1


In [72]:
CEFA_2016_EE.to_csv(root.joinpath("CEFA_2016_EE.csv"))